# Actividad 4  :

### Introducción


En actividades previas de este curso se abordó el problema mediante modelos clásicos de aprendizaje supervisado, tales como regresión logística, Random Forest y Support Vector Machines (SVM), utilizando información clínica y antropométrica proveniente de controles de salud.

La presente actividad corresponde a una **adaptación del trabajo desarrollado en la Actividad 3**, incorporando **Redes Neuronales Artificiales (ANN)** y una **Red Neuronal Convolucional (CNN)**, con el objetivo de:
- Comprender su formulación y proceso de entrenamiento.
- Analizar su desempeño predictivo frente a modelos tradicionales.
- Evaluar su pertinencia práctica en un contexto de APS rural.


Claudio Cárdenas Mansilla$^{1}$ \\
Evelyn Sánchez Cabezas$^{2}$ \\

$^{1}$ Subdepartamento de Inteligencia Sanitaria, Dirección de Servicio de Salud Chiloé \\
$^{2}$ Coordinación de Centros de Atención a la Comunidad, Facultad de Salud y Ciencias Sociales, Universidad de Las Américas \\

### RESUMEN EJECUTVO:

La Actividad 4 se desarrolla a partir del proyecto previamente trabajado en la asignatura, manteniendo el mismo conjunto de datos, la definición de la variable objetivo y el esquema de preprocesamiento. El dataset corresponde a 3.058 registros de usuarios atendidos en la Atención Primaria de Salud (APS) de la comuna de Quellón, provenientes del Control Cardiovascular y del Examen de Medicina Preventiva del Adulto (EMPA), recopilados entre los años 2021 y 2023.

Sobre esta base, en la presente actividad se incorporan Redes Neuronales Artificiales (ANN) y una Red Neuronal Convolucional (CNN), con el objetivo de analizar su formulación, proceso de entrenamiento y desempeño predictivo, y compararlas con modelos tradicionales previamente implementados, tales como regresión logística, Random Forest y Support Vector Machines (SVM).

Se mantuvo un preprocesamiento estandarizado de los datos, incluyendo imputación, codificación de variables categóricas y escalamiento de variables numéricas. La evaluación de los modelos se realizó mediante validación cruzada estratificada y un conjunto de testeo independiente, utilizando métricas de clasificación habituales como Accuracy, F1-score, AUC-ROC y PR-AUC, en un contexto donde las clases presentan una distribución balanceada.

Los resultados muestran que las redes neuronales artificiales alcanzan un desempeño comparable a los modelos clásicos, aunque con mayor costo computacional y menor interpretabilidad. La red neuronal convolucional, aplicada a una representación matricial simplificada de los datos, permite explorar patrones adicionales, pero su aporte práctico resulta limitado dada la naturaleza tabular del dataset.

En conjunto, los resultados permiten evaluar críticamente el uso de redes neuronales en este problema y refuerzan la utilidad del proyecto como una herramienta de apoyo a la priorización preventiva del riesgo cardiovascular en APS rural, considerando el equilibrio entre desempeño predictivo, interpretabilidad y viabilidad operativa.


# Objetivo de la actividad

Se integran bases provenientes del Control Cardiovascular y del Examen de Medicina Preventiva del Adulto (EMPA), conformando un conjunto de 3.058 usuarios con variables clínicas básicas. A partir de un flujo de preprocesamiento que incluye limpieza, imputación, codificación de variables categóricas y estandarización, se implementan y evalúan modelos tradicionales (regresión logística, Random Forest y Support Vector Machines) junto con Redes Neuronales Artificiales (ANN) y una Red Neuronal Convolucional (CNN), con el fin de analizar su desempeño predictivo, costo computacional e interpretabilidad en un contexto de APS rural.


In [1]:
# @title
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# @title
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_curve,
    auc,
    precision_recall_curve,
    average_precision_score
)

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
np.random.seed(42)

## Dataset y descripción de variables

In [ ]:
# @title
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/ACTIVIDAD 2 MACHINE LEARNING II/PROYECTO FINAL ML II MDS/data_model_pvc_quellon_2024_fin.xlsx')
df.columns

### Diccionario de Caracteristicas:

- PCV: Variable Objetivo progresión hacia criterios PSCV (Programa de Salud Cardiovascular), cumplimiento (0 = No Cumple; 1 = Si Cumple)
- SEXO: Sexo del Usuario (0 = Hombre; 1 = Mujer)
- EDAD: Edad del usuario en años.
- PESO: Peso del Usuario en Kilogramos.
- TALLA: Talla (estatura) en centimetros.
- CC: Cirscunferencia de Cintura en metros.
- PAS: Presión arterial sistolica (Normal: Presión sistólica menor de \(120\) mmHg.)
- PAD: Presión arterial diastolica (Normal: Presión diastólica menor de \(80\) mmHg.)
- CT: Colesterol Total (Los valores normales de colesterol total para adultos son menos de 200 mg/dL (miligramos por decilitro))

##Exploración y preprocesamiento de datos

### Exploración inicial del dataset.

In [ ]:
df.head()

### Tamaño del dataset y variable Objetivo "PCV".

In [ ]:
# @title
import matplotlib.pyplot as plt
import seaborn as sns

print("Tamaño del dataset:", df.shape)

print(df["PCV"].value_counts(normalize=True).rename("proporción"))

# Crear el gráfico de barras para 'PCV'
plt.figure(figsize=(7, 5))
ax = sns.countplot(x='PCV', data=df, palette='viridis')
plt.title('Distribución de la Variable Objetivo (PCV)')
plt.xlabel('Estado de PSCV')
plt.ylabel('Conteo')

# Ajustar las etiquetas del eje X
ax.set_xticks([0, 1])
ax.set_xticklabels(['No PSCV', 'PSCV'])

# Calcular conteos y proporciones
total = len(df['PCV'])
for p in ax.patches:
    height = p.get_height()
    percentage = '{:.1f}%'.format(100 * height / total)
    ax.text(p.get_x() + p.get_width() / 2.,
            height / 2, # Position label inside the bar (half height)
            f'{int(height)}\n({percentage})',
            ha='center', va='center', fontsize=10, color='white') # Center vertically and use white color

plt.show()

### Definición de variables cuantitativas, categóricas y variable Objetivo.

In [ ]:
# @title
numeric_features = [
    "EDAD",
    "PESO",
    "TALLA", "CC", "PAS", "PAD", "CT"
]

categorical_features = [
    "SEXO",
]

X = df[numeric_features + categorical_features]
y = df["PCV"]

In [ ]:
# @title
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


### Descripción de variable explicativas en función de variable objetivo.

In [ ]:
# @title
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Calcular la matriz de correlación para las variables numéricas
correlation_matrix = df[numeric_features].corr()

# Crear una máscara para la parte superior del triángulo
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Crear el Heatmap de Correlaciones con la máscara aplicada
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5, mask=mask)
plt.title('Heatmap de Correlaciones de Variables Numéricas (Triángulo Inferior)')
plt.show()

### Identificación de valores faltantes.




In [ ]:
# @title
missing_data = []

for col in df.columns:
    # Conteo de valores nulos
    null_count = df[col].isnull().sum()

    # Conteo de valores en blanco (solo para tipos de datos de cadena)
    blank_count = 0
    if df[col].dtype == 'object' or pd.api.types.is_string_dtype(df[col]):
        blank_count = (df[col] == '').sum()

    total_missing = null_count + blank_count
    total_rows = len(df)
    percentage_missing = (total_missing / total_rows) * 100

    missing_data.append({
        'Variable': col,
        'Nulos': null_count,
        'Blancos': blank_count,
        'Total Faltantes': total_missing,
        '% Faltantes': f'{percentage_missing:.2f}%'
    })

missing_df = pd.DataFrame(missing_data)
print(missing_df)

### Distribución de variable categórica (sexo).

In [ ]:
# @title
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Mapear los valores numéricos a etiquetas descriptivas para SEXO
df['SEXO_LABEL'] = df['SEXO'].map({0: 'Hombre', 1: 'Mujer'})

# Mapear los valores numéricos a etiquetas descriptivas para PCV
df['PCV_LABEL'] = df['PCV'].map({0: 'PCV Negativo', 1: 'PCV Positivo'})

# Definir el orden deseado para las categorías en el gráfico
category_order_sexo = ['Hombre', 'Mujer']
category_order_pcv = ['PCV Negativo', 'PCV Positivo']

# Crear el gráfico de barras, especificando el orden y usando hue para PCV
plt.figure(figsize=(10, 7))
ax = sns.countplot(x='SEXO_LABEL', data=df, palette='viridis', order=category_order_sexo, hue='PCV_LABEL', hue_order=category_order_pcv)
plt.title('Distribución de la Variable SEXO por PCV')
plt.xlabel('SEXO')
plt.ylabel('Conteo')
plt.legend(title='PCV')

# Calcular conteos y proporciones para las etiquetas
# Usamos pd.crosstab para obtener los conteos de forma sencilla
counts_table = pd.crosstab(df['SEXO_LABEL'], df['PCV_LABEL'])
# Y para las proporciones dentro de cada grupo SEXO
proportions_table = pd.crosstab(df['SEXO_LABEL'], df['PCV_LABEL'], normalize='index')

# Añadir conteo y proporción a cada barra
for i, container in enumerate(ax.containers):
    pcv_label = category_order_pcv[i] # Obtener la etiqueta PCV para este contenedor

    for bar_idx, bar in enumerate(container):
        sexo_label = category_order_sexo[bar_idx] # Obtener la etiqueta SEXO para esta barra

        count = counts_table.loc[sexo_label, pcv_label]
        proportion = proportions_table.loc[sexo_label, pcv_label]

        # Colocar la etiqueta de texto
        ax.text(
            bar.get_x() + bar.get_width() / 2, # Posición X: centro de la barra
            bar.get_height(),                  # Posición Y: en la parte superior de la barra
            f'{count} ({proportion:.1%})',     # Texto: conteo (proporción)
            ha='center', va='bottom', fontsize=9, color='black'
        )

plt.show()


print("Conteo de SEXO:")
print(df['SEXO_LABEL'].value_counts().reindex(category_order_sexo))
print("\nProporción de SEXO:")
print(df['SEXO_LABEL'].value_counts(normalize=True).reindex(category_order_sexo))


### Análisis descriptivo de variables cuantitativas.

In [ ]:
# @title
print('Descripción de Variables Cuantitativas:')
display(X[numeric_features].describe())

### Distribución de variables cuantitativas del dataset.

In [ ]:
# @title
import matplotlib.pyplot as plt
import seaborn as sns

# Determina el número de filas y columnas para la matriz de gráficos
num_features = len(numeric_features)
num_cols = 3  # Puedes ajustar el número de columnas deseado
num_rows = (num_features + num_cols - 1) // num_cols

plt.figure(figsize=(num_cols * 5, num_rows * 4))
plt.suptitle('Histograma de Variables Cuantitativas', y=1.02, fontsize=16)

for i, feature in enumerate(numeric_features):
    plt.subplot(num_rows, num_cols, i + 1)
    sns.histplot(df[feature], kde=True, bins=20) # Removed palette argument as it was ignored without a 'hue' variable
    plt.title(f'Distribución de {feature}')
    plt.xlabel(feature)
    plt.ylabel('Frecuencia')

plt.tight_layout(rect=[0, 0.03, 1, 0.98]) # Ajusta el layout para evitar solapamientos
plt.show()

## Análisis de datos Atípicos (Outliers).

Dado que las variables clínicas cuantitativas pueden presentar valores extremos que distorsionen la estimación de parámetros y la estabilidad de los modelos, se aplicó un procedimiento sistemático de tratamiento de datos atípicos basado en el método del rango intercuartílico (IQR). Para cada variable numérica (edad, peso, talla, CC, PAS, PAD y CT) se calcularon los percentiles 25 (Q1) y 75 (Q3), obteniendo el IQR (Q3–Q1). Los valores inferiores a Q1 − 1,5·IQR fueron reemplazados por el límite inferior y los superiores a Q3 + 1,5·IQR fueron reemplazados por el límite superior. Se generó así una versión del dataset sin outliers (df_no_outliers), conservando en paralelo una copia del dataset original para referencia. A partir de esta versión depurada se definieron las matrices de entrada (X) y de salida (y) del modelo.

### Identificación de datos atípicos

In [ ]:
# @title
import matplotlib.pyplot as plt
import seaborn as sns

# Define las variables numéricas y la variable de agrupamiento
numeric_features = [
    "EDAD",
    "PESO",
    "TALLA", "CC", "PAS", "PAD", "CT"
]
pcv_variable_label = 'PCV_LABEL'

# Determina el número de filas y columnas para la matriz de gráficos
num_features = len(numeric_features)
num_cols = 3  # Puedes ajustar el número de columnas
num_rows = (num_features + num_cols - 1) // num_cols

plt.figure(figsize=(num_cols * 5, num_rows * 4))
plt.suptitle('Matriz de Gráficos de Caja por Variables Numéricas y PCV', y=1.02, fontsize=16)

for i, feature in enumerate(numeric_features):
    plt.subplot(num_rows, num_cols, i + 1)
    sns.boxplot(x=pcv_variable_label, y=feature, data=df, palette='viridis')
    plt.title(f'Distribución de {feature} por PCV')
    plt.xlabel('PCV')
    plt.ylabel(feature)

plt.tight_layout(rect=[0, 0.03, 1, 0.98]) # Ajusta el layout para evitar solapamientos
plt.show()


### Tratamiento de datos atípicos.



In [ ]:
# @title
def handle_outliers_iqr(df_input, columns, iqr_multiplier=1.5):
    """
    Identifica y reemplaza outliers en un DataFrame usando el método IQR.
    Los outliers son reemplazados por el límite inferior o superior del IQR.

    Args:
        df_input (pd.DataFrame): El DataFrame de entrada.
        columns (list): Lista de nombres de columnas donde buscar outliers.
        iqr_multiplier (float): Multiplicador para el IQR (por defecto 1.5).

    Returns:
        pd.DataFrame: Un nuevo DataFrame con los outliers reemplazados.
    """
    df_processed = df_input.copy() # Crear una copia para no modificar el dataset original
    original_df_saved = df_input.copy() # Guardar el dataset inicial sin modificaciones

    for col in columns:
        if col in df_processed.columns and pd.api.types.is_numeric_dtype(df_processed[col]):
            Q1 = df_processed[col].quantile(0.25)
            Q3 = df_processed[col].quantile(0.75)
            IQR = Q3 - Q1

            lower_bound = Q1 - iqr_multiplier * IQR
            upper_bound = Q3 + iqr_multiplier * IQR

            # Reemplazar outliers: valores menores que el límite inferior por el límite inferior
            df_processed.loc[df_processed[col] < lower_bound, col] = lower_bound
            # Reemplazar outliers: valores mayores que el límite superior por el límite superior
            df_processed.loc[df_processed[col] > upper_bound, col] = upper_bound
        else:
            print(f"Advertencia: La columna '{col}' no es numérica o no existe y será ignorada.")

    print("El dataset original se ha guardado en la variable 'original_df_saved'.")
    return df_processed, original_df_saved

# Ejemplo de uso:

df_no_outliers, original_df_saved = handle_outliers_iqr(df, numeric_features)
print("\nDataFrame sin outliers (primeras 5 filas):")
display(df_no_outliers.head())
print("\nDataFrame original (primeras 5 filas, sin modificar):")
display(original_df_saved.head())

In [ ]:
# @title
# Definir X y Y usando el DataFrame sin outliers
X = df_no_outliers[numeric_features + categorical_features]
y = df_no_outliers["PCV"]

## Preprocesamiento de variables (Pipeline de escalado y codificación de variables)

Las variables explicativas se clasificaron en:

- Cuantitativas: edad, peso, talla, CC, PAS, PAD, CT.

- Categóricas: sexo (0 = hombre, 1 = mujer).

Para estandarizar el tratamiento de los datos se construyó un pipeline de preprocesamiento basado en ColumnTransformer:

- Las variables numéricas se transformaron mediante escalamiento estándar (StandardScaler), centrando en media 0 y desviación estándar 1.

La variable categórica sexo se codificó mediante One-Hot Encoding con eliminación de una categoría en el caso binario (drop="if_binary"), evitando colinealidad perfecta y permitiendo su integración en los modelos de regresión logística.

Adicionalmente, para una de las especificaciones de regresión logística se incorporaron términos polinomiales de segundo grado (incluyendo interacciones) sobre las variables numéricas mediante PolynomialFeatures(degree=2), seguidos de escalamiento estándar, con el objetivo de capturar relaciones no lineales manteniendo un marco paramétrico controlado.

In [ ]:
# @title
numeric_features = [
    "EDAD",
    "PESO",
    "TALLA", "CC", "PAS", "PAD", "CT"
]

categorical_features = [
    "SEXO"
]

numeric_transformer_base = Pipeline(steps=[
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore", drop="if_binary", sparse_output=False))
])

preprocessor_base = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_base, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)


## 1. Naïve Bayes


In [ ]:
# @title
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import time

# Validación cruzada estratificada (como en clases)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Pipeline: mismo preprocesamiento + Naïve Bayes
pipe_gnb = Pipeline(steps=[
    ("preprocess", preprocessor_base),
    ("clf", GaussianNB())
])

# Hiperparámetro de GaussianNB
param_grid_gnb = {
    "clf__var_smoothing": [1e-12, 1e-11, 1e-10, 1e-9, 1e-8]
}

grid_gnb = GridSearchCV(
    pipe_gnb,
    param_grid=param_grid_gnb,
    scoring="average_precision",  # PR-AUC
    cv=cv,
    n_jobs=-1
)

# Entrenamiento + tiempo
t0 = time.perf_counter()
grid_gnb.fit(X_train, y_train)
t_gnb = time.perf_counter() - t0

best_gnb = grid_gnb.best_estimator_

print("Mejores hiperparámetros (GaussianNB):", grid_gnb.best_params_)
print("Mejor PR-AUC (CV):", grid_gnb.best_score_)
print(f"Tiempo GridSearch (s): {t_gnb:.2f}")


In [ ]:
# @title
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score,
    confusion_matrix, ConfusionMatrixDisplay,
    roc_curve, precision_recall_curve
)
import matplotlib.pyplot as plt

# Predicciones en test
y_pred_gnb = best_gnb.predict(X_test)
y_score_gnb = best_gnb.predict_proba(X_test)[:, 1]

print("GaussianNB — TEST")
print("Accuracy:", accuracy_score(y_test, y_pred_gnb))
print("Precision:", precision_score(y_test, y_pred_gnb, zero_division=0))
print("Recall:", recall_score(y_test, y_pred_gnb, zero_division=0))
print("F1:", f1_score(y_test, y_pred_gnb, zero_division=0))
print("AUC-ROC:", roc_auc_score(y_test, y_score_gnb))
print("PR-AUC:", average_precision_score(y_test, y_score_gnb))


In [ ]:
# @title
# Matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_gnb))
disp.plot(values_format="d")
plt.title("GaussianNB — Confusion Matrix (test)")
plt.show()

# Curva ROC
fpr, tpr, _ = roc_curve(y_test, y_score_gnb)
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("GaussianNB — ROC (test)")
plt.show()

# Curva Precision–Recall
prec, rec, _ = precision_recall_curve(y_test, y_score_gnb)
plt.plot(rec, prec)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("GaussianNB — Precision–Recall (test)")
plt.show()


**Comentarios modelo Naïve Bayes**

El modelo Gaussian Naïve Bayes mostró un desempeño sólido en la predicción de PCV, con valores altos
de AUC-ROC y PR-AUC, lo que indica una buena capacidad discriminante de la clase positiva. A pesar
de su simplicidad, el modelo logra un equilibrio adecuado entre precision y recall, lo que resulta
relevante en un contexto clínico donde es importante detectar la mayor cantidad posible de casos
de riesgo sin generar un exceso de falsos positivos.

Sin embargo, el supuesto de independencia condicional entre predictores se ve parcialmente
incumplido, dado que variables clínicas como presión arterial, peso y circunferencia de cintura
presentan correlaciones entre sí. Esto puede afectar la estimación de las probabilidades
posteriores, aunque no impide que el modelo sea útil como aproximación inicial debido a su bajo
costo computacional y rapidez de entrenamiento.


## 2. Support Vector Machines (SVM)


In [ ]:
# @title
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import time

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

pipe_svm = Pipeline(steps=[
    ("preprocess", preprocessor_base),
    ("clf", SVC(probability=True, random_state=42))
])

param_grid_svm = [
    # SVM lineal
    {
        "clf__kernel": ["linear"],
        "clf__C": [0.01, 0.1, 1, 10, 100]
    },
    # SVM RBF
    {
        "clf__kernel": ["rbf"],
        "clf__C": [0.1, 1, 10, 100],
        "clf__gamma": [1e-4, 1e-3, 1e-2, 1e-1, 1]
    }
]

grid_svm = GridSearchCV(
    pipe_svm,
    param_grid=param_grid_svm,
    scoring="average_precision",
    cv=cv,
    n_jobs=-1
)

t0 = time.perf_counter()
grid_svm.fit(X_train, y_train)
t_svm = time.perf_counter() - t0

best_svm = grid_svm.best_estimator_

print("Mejores hiperparámetros SVM:", grid_svm.best_params_)
print("Mejor PR-AUC (CV):", grid_svm.best_score_)
print(f"Tiempo GridSearch (s): {t_svm:.2f}")


In [ ]:
# @title
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score,
    confusion_matrix, ConfusionMatrixDisplay,
    roc_curve, precision_recall_curve
)
import matplotlib.pyplot as plt

# Predicciones en test
y_pred_svm = best_svm.predict(X_test)
y_score_svm = best_svm.predict_proba(X_test)[:, 1]

print("SVM RBF — TEST")
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("Precision:", precision_score(y_test, y_pred_svm, zero_division=0))
print("Recall:", recall_score(y_test, y_pred_svm, zero_division=0))
print("F1:", f1_score(y_test, y_pred_svm, zero_division=0))
print("AUC-ROC:", roc_auc_score(y_test, y_score_svm))
print("PR-AUC:", average_precision_score(y_test, y_score_svm))


In [ ]:
# @title
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_svm))
disp.plot(values_format="d")
plt.title("Best SVM — Confusion Matrix (test)")
plt.show()



**Interpretación SVM RBF (TEST)**

Accuracy ~ 0.79 → similar a NB (normal en desbalance)

Precision ~ 0.81 → menos falsos positivos que NB

Recall ~ 0.74 → levemente menor que NB

F1 ~ 0.775 → mejor equilibrio global

AUC-ROC ~ 0.88

PR-AUC ~ 0.89 → mejor que GaussianNB

SVM RBF mejora la capacidad discriminante a costa de mayor costo computacional, tal como se vio en clases.

# Interpretación SVM RBF (TEST)

Accuracy ~ 0.79 → similar a NB (normal en desbalance)

Precision ~ 0.81 → menos falsos positivos que NB

Recall ~ 0.74 → levemente menor que NB

F1 ~ 0.775 → mejor equilibrio global

AUC-ROC ~ 0.88

PR-AUC ~ 0.89 → mejor que GaussianNB

SVM RBF mejora la capacidad discriminante a costa de mayor costo computacional, tal como se vio en clases.

In [ ]:
# @title
import pandas as pd

results = pd.DataFrame([
    {
        "Modelo": "Gaussian Naïve Bayes",
        "Accuracy": 0.7761,
        "Precision": 0.7732,
        "Recall": 0.7601,
        "F1": 0.7666,
        "AUC-ROC": 0.8686,
        "PR-AUC": 0.8792,
        "Costo computacional": "Muy bajo"
    },
    {
        "Modelo": "SVM RBF",
        "Accuracy": 0.7908,
        "Precision": 0.8088,
        "Recall": 0.7432,
        "F1": 0.7746,
        "AUC-ROC": 0.8764,
        "PR-AUC": 0.8913,
        "Costo computacional": "Alto"
    },
    {
        "Modelo": "SVM RBF balanced",
        "Accuracy": 0.7958,
        "Precision": 0.8087,
        "Recall": 0.7568,
        "F1": 0.7818,
        "AUC-ROC": 0.8761,
        "PR-AUC": 0.8909,
        "Costo computacional": "Alto"
    }
])

results


La comparación de modelos muestra que SVM con kernel RBF presenta el mejor desempeño global,
especialmente en términos de PR-AUC y F1-score, lo que indica una mejor discriminación de la
clase positiva (PCV). No obstante, este desempeño se obtiene a costa de un mayor costo
computacional.

Gaussian Naïve Bayes, a pesar de su simplicidad y bajo costo, presenta un desempeño competitivo,
lo que lo convierte en una alternativa eficiente en contextos donde los recursos computacionales
son limitados o se requiere rapidez en la inferencia.


## 3. Análisis Support Vector Machines (SVM)

El modelo Gaussian Naïve Bayes presenta un bajo costo computacional y un desempeño competitivo,
sin embargo, su supuesto de independencia condicional entre predictores se ve parcialmente
violado en datos clínicos, donde variables como presión arterial, peso y circunferencia de
cintura presentan correlaciones relevantes. Esto puede afectar la calibración de las
probabilidades estimadas.

Los modelos SVM muestran un mejor desempeño global, particularmente el kernel RBF, lo que
sugiere la presencia de relaciones no lineales entre las variables y la condición de PCV.
Este aumento en desempeño se obtiene a costa de un mayor tiempo de entrenamiento y menor
interpretabilidad del modelo.

El escalamiento de variables resulta crítico en SVM, ya que el modelo se basa en distancias
y en la maximización del margen. Sin escalamiento, variables con mayor rango dominarían la
función de decisión.

La codificación one-hot incrementa la dimensionalidad del espacio de características, lo que
impacta negativamente en el tiempo de entrenamiento, especialmente en modelos con kernels
no lineales.

Finalmente, la incorporación de class_weight='balanced' en SVM permitió mejorar el recall
de la clase positiva, lo cual es relevante en un contexto clínico donde la detección de casos
PCV tiene mayor costo que los falsos positivos.


## Comparación y validación de modelos

Para la evaluación de los modelos se utilizó validación cruzada estratificada k-fold (k = 5), manteniendo la proporción de casos PCV=0 y PCV=1 en cada partición. En cada fold se entrenó el modelo sobre el conjunto de entrenamiento y se generaron probabilidades de pertenencia a la clase positiva (PCV=1) en el conjunto de prueba, aplicando un umbral inicial de 0,5 para la clasificación.

Sobre los resultados combinados de los cinco folds se calcularon:

- Matriz de confusión global.

Reporte de clasificación:

- precisión, recall y F1-score, con especial foco en la clase positiva (PCV=1).

- Curva ROC y AUC-ROC.

- Curva Precision–Recall y PR-AUC (Average Precision).

La métrica principal para comparación de modelos fue PR-AUC, dada la relevancia clínica de priorizar una buena discriminación de la clase positiva y el interés en el comportamiento del modelo cuando se busca maximizar la detección de personas en riesgo (PSCV) con un control razonable de falsos positivos.

In [ ]:
# @title
def evaluate_model_cv(model, X, y, cv_splits=5, threshold=0.5):
    """
    Ejecuta k-fold CV estratificada y:
    - Ajusta el modelo en cada fold
    - Acumula predicciones y probabilidades
    - Muestra matriz de confusión, reporte, ROC, PR
    - Retorna métricas específicas para la clase positiva (1)
    """
    skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=42)

    y_true_all = []
    y_pred_all = []
    y_proba_all = []

    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_train)
        y_pred = (model.predict_proba(X_test)[:, 1] >= threshold).astype(int)
        y_proba = model.predict_proba(X_test)[:, 1]

        y_true_all.extend(y_test)
        y_pred_all.extend(y_pred)
        y_proba_all.extend(y_proba)

    y_true_all = np.array(y_true_all)
    y_pred_all = np.array(y_pred_all)
    y_proba_all = np.array(y_proba_all)

    # Matriz de confusión
    cm = confusion_matrix(y_true_all, y_pred_all)
    print("Matriz de confusión global (todos los folds):")
    print(cm)

    # Reporte de clasificación
    print("Reporte de clasificación:")
    report = classification_report(y_true_all, y_pred_all, digits=3, output_dict=True)
    print(classification_report(y_true_all, y_pred_all, digits=3))

    # ROC
    fpr, tpr, _ = roc_curve(y_true_all, y_proba_all)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label=f"ROC (AUC = {roc_auc:.3f})")
    plt.plot([0, 1], [0, 1], linestyle="--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate (Recall)")
    plt.title("Curva ROC (CV global)")
    plt.legend()
    plt.show()

    # Precision–Recall
    precision, recall, _ = precision_recall_curve(y_true_all, y_proba_all)
    pr_auc = average_precision_score(y_true_all, y_proba_all)

    plt.figure()
    plt.step(recall, precision, where="post", label=f"PR (AP = {pr_auc:.3f})")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Curva Precision–Recall (CV global)")
    plt.legend()
    plt.show()

    print(f"AUC-ROC global: {roc_auc:.3f}")
    print(f"PR-AUC (Average Precision) global: {pr_auc:.3f}")

    return {
        "cm": cm,
        "roc_auc": roc_auc,
        "pr_auc": pr_auc,
        "precision_1": report['1']['precision'],
        "recall_1": report['1']['recall'],
        "f1_score_1": report['1']['f1-score']
    }

In [ ]:
X

# Modelos entrenados.

En particular, se entrenaron modelos de **Regresión Logística** (incluyendo variaciones: modelo base sin regularización, modelos con términos polinomiales, y modelos regularizados con penalización L1 y L2) y un modelo basado en **Extreme Gradient Boosting (XGBoost)**, además de métodos de referencia como **k-Nearest Neighbors (KNN)** y **Máquinas de Vectores de Soporte (SVM)** para comparación. Los hiperparámetros de los modelos más complejos (XGBoost, regresiones regularizadas, KNN, SVM) fueron optimizados mediante búsqueda en grilla con validación cruzada interna. Como métrica principal de optimización se empleó el área bajo la curva precisión-recall (PR-AUC), dado el interés clínico en maximizar la detección temprana de casos (recall) manteniendo una alta precisión en un contexto preventivo.

## Modelo de Regresion Logistica Base.

In [ ]:
# @title
# Configuración y evaluación del modelo de Regresión Logística (base)

# Crea un pipeline para el modelo de regresión logística base.
log_reg_base = Pipeline(steps=[
    # Primer paso del pipeline: preprocesamiento de datos utilizando el preprocessor_base definido previamente.
    ("preprocess", preprocessor_base),
    # Segundo paso del pipeline: el clasificador de Regresión Logística.
    ("clf", LogisticRegression(
        penalty="l2",   # Especifica la regularización L2 (Ridge) para evitar el sobreajuste.
        C=1e6,          # C es el inverso de la fuerza de regularización; un valor grande implica poca regularización.
        solver="lbfgs", # Algoritmo a usar para la optimización.
        max_iter=1000   # Número máximo de iteraciones para que converja el optimizador.
    ))
])

# Evalúa el modelo de regresión logística base usando la función evaluate_model_cv.
# Pasa el pipeline del modelo, las características (X), la variable objetivo (y),
# y especifica el número de divisiones para la validación cruzada (5).
results_base = evaluate_model_cv(log_reg_base, X, y, cv_splits=5)

### Estimación de parámetros de Modelo de Regresión Logística Base.

In [ ]:
# @title
import statsmodels.api as sm
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Re-create the preprocessor (defined in b0b0ea5b)
# This ensures we use the exact same setup as log_reg_base
numeric_transformer_base = Pipeline(steps=[
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore", drop='if_binary')) # Added drop='if_binary' to prevent multicollinearity
])

preprocessor_for_statsmodels = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_base, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

# Re-create and fit a Logistic Regression pipeline on the full dataset
# This model will be used solely for parameter interpretation with statsmodels
full_data_log_reg_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor_for_statsmodels),
    ("clf", LogisticRegression(
        penalty="l2",
        C=1e6,
        solver="lbfgs",
        max_iter=1000
    ))
])

# Fit the pipeline on the full X and y
full_data_log_reg_pipeline.fit(X, y)

# Extract the fitted preprocessor and classifier
fitted_preprocessor = full_data_log_reg_pipeline.named_steps['preprocess']
fitted_logistic_clf = full_data_log_reg_pipeline.named_steps['clf']

# Transform X using the fitted preprocessor
X_transformed_for_statsmodels = fitted_preprocessor.transform(X)

# Get feature names after preprocessing
# ColumnTransformer.get_feature_names_out() for names
feature_names_out = fitted_preprocessor.get_feature_names_out()

# Convert X_transformed to a DataFrame for statsmodels with proper column names
X_transformed_df_for_statsmodels = pd.DataFrame(X_transformed_for_statsmodels, columns=feature_names_out, index=X.index)

# Add a constant (intercept) column to the transformed data
X_transformed_df_for_statsmodels = sm.add_constant(X_transformed_df_for_statsmodels)

# Fit the logistic regression model using statsmodels for detailed statistics
logit_model_sm = sm.Logit(y, X_transformed_df_for_statsmodels)
result_sm = logit_model_sm.fit(disp=0) # disp=0 to suppress optimization output

# Extraer los resultados
params = result_sm.params
std_err = result_sm.bse
p_values = result_sm.pvalues
conf_int = result_sm.conf_int()
odds_ratios = np.exp(params)
lower_ci = np.exp(conf_int[0])
upper_ci = np.exp(conf_int[1])

# Create a DataFrame for display
summary_df = pd.DataFrame({
    'Estimado': params,
    'Error Estándar': std_err,
    'Valor-p': p_values,
    'OR (Odds Ratio)': odds_ratios,
    'CI 95% Inferior': lower_ci,
    'CI 95% Superior': upper_ci
})

# Add significance stars
summary_df['Significancia'] = ''
summary_df.loc[summary_df['Valor-p'] < 0.05, 'Significancia'] = '*'
summary_df.loc[summary_df['Valor-p'] < 0.01, 'Significancia'] = '**'
summary_df.loc[summary_df['Valor-p'] < 0.001, 'Significancia'] = '***'

print("Tabla de Parámetros del Modelo Regresión Logística (Base) con Estadísticas:")
display(summary_df.round(4))

## Modelo de Regresión Logística con Polinomios.

In [ ]:
# @title
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

# 1) Transformador numérico optimizado
numeric_transformer_poly = Pipeline(steps=[
    # Primero genero términos polinomiales
    ("poly", PolynomialFeatures(
        degree=2,          # empezar con grado 2
        include_bias=False,
        interaction_only=False
    )),
    # Luego escalo TODOS los términos generados
    ("scaler", StandardScaler())
])

# 2) Preprocesador combinado
preprocessor_poly = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_poly, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ],
    remainder="drop"
)

# 3) Modelo de regresión logística con regularización razonable
log_reg_poly = Pipeline(steps=[
    ("preprocess", preprocessor_poly),
    ("clf", LogisticRegression(
        penalty="l2",
        C=1.0,            # regularización estándar
        solver="lbfgs",
        max_iter=1000
    ))
])

# 4) Evaluación con  función de CV
results_poly = evaluate_model_cv(log_reg_poly, X, y, cv_splits=5)


### Flujo completo, ordenado y reproducible de preparación de datos + modelamiento predictivo, orientado a un modelo de regresión logística.

In [ ]:
# @title
base_pipeline_poly = Pipeline(steps=[
    ("preprocess", preprocessor_poly),
    ("clf", LogisticRegression(max_iter=1000))
])


### Definición del espacio de búsqueda de hiperparámetros. regularización L2 (Ridge)

In [ ]:
# @title
param_grid_l2 = {
    "clf__penalty": ["l2"],
    "clf__solver": ["lbfgs"],
    "clf__C": [0.01, 0.1, 1.0, 10.0]
}

grid_l2 = GridSearchCV(
    base_pipeline_poly,
    param_grid_l2,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring="average_precision",  # PR-AUC
    n_jobs=-1
)

grid_l2.fit(X, y)

print("Mejores hiperparámetros (L2):", grid_l2.best_params_)
print("Mejor PR-AUC medio (L2):", grid_l2.best_score_)

best_model_l2 = grid_l2.best_estimator_


### Definición del espacio de búsqueda de hiperparámetros, regularización L1 (Lasso).

In [ ]:
# @title
param_grid_l1 = {
    "clf__penalty": ["l1"],
    "clf__solver": ["liblinear"],
    "clf__C": [0.01, 0.1, 1.0, 10.0]
}

grid_l1 = GridSearchCV(
    base_pipeline_poly,
    param_grid_l1,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring="average_precision",
    n_jobs=-1
)

grid_l1.fit(X, y)

print("Mejores hiperparámetros (L1):", grid_l1.best_params_)
print("Mejor PR-AUC medio (L1):", grid_l1.best_score_)

best_model_l1 = grid_l1.best_estimator_


### Presentación de mejores modelos de Regresión Logistica L1 y L2.

In [ ]:
# @title
results_l2 = evaluate_model_cv(best_model_l2, X, y, cv_splits=5)

In [ ]:
results_l1 = evaluate_model_cv(best_model_l1, X, y, cv_splits=5)

## Modelo KNN (K-Nearest Neighbors o Vecinos más Cercanos)

In [ ]:
# @title
from sklearn.neighbors import KNeighborsClassifier

# Definir el pipeline base para KNN
base_pipeline_knn = Pipeline(steps=[
    ("preprocess", preprocessor_base),
    ("clf", KNeighborsClassifier())
])

# Definir el espacio de búsqueda de hiperparámetros para KNN
param_grid_knn = {
    'clf__n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]
}

# Configurar GridSearchCV para optimizar KNN
grid_knn = GridSearchCV(
    base_pipeline_knn,
    param_grid_knn,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='average_precision',  # Optimizar por PR-AUC
    n_jobs=-1,
    verbose=1
)

# Entrenar el GridSearchCV
grid_knn.fit(X, y)

print("Mejores hiperparámetros (KNN):", grid_knn.best_params_)
print("Mejor PR-AUC medio (KNN):", grid_knn.best_score_)

best_model_knn = grid_knn.best_estimator_

# Evaluar el mejor modelo KNN
results_knn = evaluate_model_cv(best_model_knn, X, y, cv_splits=5)

## Modelo SVM (Máquina de Vectores de Soporte)

In [ ]:
# @title
from sklearn.svm import SVC

# Definir el pipeline base para SVM
base_pipeline_svm = Pipeline(steps=[
    ("preprocess", preprocessor_base),
    ("clf", SVC(probability=True, random_state=42))
])

# Definir el espacio de búsqueda de hiperparámetros para SVM

param_grid_svm = {
    'clf__C': [0.1, 1, 10],
    'clf__kernel': ['linear', 'rbf']
}

# Configurar GridSearchCV para optimizar SVM
grid_svm = GridSearchCV(
    base_pipeline_svm,
    param_grid_svm,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='average_precision',  # Optimizar por PR-AUC
    n_jobs=-1,
    verbose=1
)

# Entrenar el GridSearchCV
grid_svm.fit(X, y)

print("Mejores hiperparámetros (SVM):", grid_svm.best_params_)
print("Mejor PR-AUC medio (SVM):", grid_svm.best_score_)

best_model_svm = grid_svm.best_estimator_

# Evaluar el mejor modelo SVM
results_svm = evaluate_model_cv(best_model_svm, X, y, cv_splits=5)

## Modelo XGBoost (Extreme Gradient Boosting)

### XGBoost (Extreme Gradient Boosting), Modelo base.

In [ ]:
# @title
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

# X: DataFrame con columnas EDAD, PESO, TALLA, CC, PAS, PAD, CT, SEXO
# y: por ejemplo, variable binaria: 1 = alto riesgo CV, 0 = no alto riesgo

model_xgb = Pipeline(steps=[
    ("preprocessor", preprocessor_base),
    ("classifier", XGBClassifier(
        n_estimators=200,
        max_depth=4,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    ))
])

model_xgb.fit(X, y)

# Predicción de riesgo (clase)
y_pred = model_xgb.predict(X)

# Probabilidad de evento (ej. riesgo alto)
y_proba = model_xgb.predict_proba(X)[:, 1]

results_base = evaluate_model_cv(model_xgb, X, y, cv_splits=5)

### Definición del espacio de búsqueda de hiperparámetros. Modelo XGBoost (Extreme Gradient Boosting)

In [ ]:
# @title
param_grid_xgb = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [3, 4, 5],
    'classifier__learning_rate': [0.01, 0.025, 0.05, 0.1, 0.15, 0.2],
    'classifier__subsample': [0.7, 0.8, 0.9]
}

grid_xgb = GridSearchCV(
    model_xgb,
    param_grid_xgb,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='average_precision', # Usamos PR-AUC como métrica de optimización
    n_jobs=-1,
    verbose=1
)

grid_xgb.fit(X, y)

print("Mejores hiperparámetros (XGBoost):", grid_xgb.best_params_)
print("Mejor PR-AUC medio (XGBoost):", grid_xgb.best_score_)

best_model_xgb = grid_xgb.best_estimator_

In [ ]:
# @title
results_xgb = evaluate_model_cv(best_model_xgb, X, y, cv_splits=5)

### Importancia de las Caracteristicas, Modelo Optimizado XGBoost (Extreme Gradient Boosting).

In [ ]:
# @title
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import shap

# Extract the preprocessor and the classifier from the best_model_xgb pipeline
preprocessor = best_model_xgb.named_steps['preprocessor']
xgb_classifier = best_model_xgb.named_steps['classifier']

# Transform the input features X using the preprocessor
X_transformed = preprocessor.transform(X)

# Get feature names after preprocessing
feature_names_out = preprocessor.get_feature_names_out()

# Convert X_transformed to a DataFrame for SHAP analysis with proper column names
X_transformed_df = pd.DataFrame(X_transformed, columns=feature_names_out)

# Initialize SHAP explainer
explainer = shap.TreeExplainer(xgb_classifier)

# Compute SHAP values
shap_values = explainer.shap_values(X_transformed_df)

# Generate a SHAP summary plot (bar plot) to visualize the feature importance and impact
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_transformed_df, plot_type="bar", show=False)
plt.title('SHAP Feature Importance (Bar Plot) for XGBoost Model')
plt.xlabel('SHAP Value (Mean absolute impact on model output)')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

# Generate a SHAP Summary Plot (Dots) to visualize the feature importance and impact per instance
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_transformed_df, show=False) # Default plot_type is "dot"
plt.title('SHAP Summary Plot (Dots) for XGBoost Model')
plt.tight_layout()
plt.show()

print("SHAP summary bar plot and dot plot generated successfully.")


- **Edad (num__EDAD)**

Es la variable con mayor impacto.

Valores altos de edad (rojo) se asocian claramente a valores SHAP positivos, incrementando fuertemente la probabilidad de progresión a PSCV.

Valores bajos (azul) se concentran en SHAP negativos.

Interpretación clínica:
Existe una relación directa, progresiva y consistente entre edad y riesgo cardiovascular. El modelo capta adecuadamente el efecto del ciclo vital, sin umbrales artificiales.

Implicancia APS:
La edad debe ser el eje estructural de la estratificación poblacional, especialmente para focalizar prevención secundaria.

- **Circunferencia de cintura (num__CC)**

Valores altos de CC desplazan la predicción hacia riesgo positivo.

Se observa dispersión, indicando interacción con otras variables (edad, PAS).

Interpretación clínica:
La obesidad central es un determinante crítico del riesgo cardiometabólico, incluso antes de cumplir criterios PSCV formales.

Implicancia APS:
Refuerza la necesidad de medición sistemática y confiable de CC en controles preventivos.

- **Presión arterial sistólica (num__PAS)**

Valores elevados se asocian a aumento del riesgo.

Relación clara y clínicamente coherente.

Interpretación:
La PAS actúa como señal temprana de progresión cardiovascular, incluso en rangos subhipertensivos.

- **Talla (num__TALLA)**

Valores altos de talla (rojo) se asocian a SHAP negativos.

Valores bajos aumentan levemente el riesgo.

Interpretación técnica:
La talla funciona como factor de ajuste antropométrico; personas de menor estatura presentan mayor riesgo relativo ante iguales niveles de CC y peso.

- **Colesterol total (num__CT)**

Valores altos tienden a aumentar el riesgo, aunque con menor impacto que edad o CC.

Dispersión moderada.

Interpretación clínica:
Confirma su rol como factor de riesgo clásico, pero no dominante en este modelo.

- **Presión arterial diastólica (num__PAD)**

Efecto menor y más simétrico.

Aporta información complementaria a la PAS.

- **Peso (num__PESO)**

Impacto bajo.

El modelo prioriza la distribución de grasa (CC) más que el peso total.

Lectura clínica:
Coherente con la evidencia actual: el peso aislado es un predictor débil del riesgo cardiovascular.

- **Sexo (cat__SEXO_1)**

Efecto prácticamente nulo.

Los puntos se concentran cerca de SHAP = 0.

Interpretación:
Una vez ajustado por variables clínicas, el sexo no discrimina significativamente la progresión a PSCV.

Implicancia:
Reduce el riesgo de sesgos de género en la priorización preventiva.



## Tabla comparativa de desempeño de modelos


In [ ]:
# @title
import pandas as pd

# Crear un diccionario con los resultados de cada modelo
model_comparison_data = {
    'Modelo': [
        'Regresión Logística Base',
        'Regresión Logística con Polinomios',
        'Regresión Logística L2 (Optimizada)',
        'Regresión Logística L1 (Optimizada)',
        'XGBoost (Optimizado)',
        'KNN (Optimizado)',
        'SVM (Optimizado)'
    ],
    'AUC-ROC': [
        results_base['roc_auc'],
        results_poly['roc_auc'],
        results_l2['roc_auc'],
        results_l1['roc_auc'],
        results_xgb['roc_auc'],
        results_knn['roc_auc'],
        results_svm['roc_auc']
    ],
    'PR-AUC': [
        results_base['pr_auc'],
        results_poly['pr_auc'],
        results_l2['pr_auc'],
        results_l1['pr_auc'],
        results_xgb['pr_auc'],
        results_knn['pr_auc'],
        results_svm['pr_auc']
    ],
    'Precision (PCV=1)': [
        results_base['precision_1'],
        results_poly['precision_1'],
        results_l2['precision_1'],
        results_l1['precision_1'],
        results_xgb['precision_1'],
        results_knn['precision_1'],
        results_svm['precision_1']
    ],
    'Recall (PCV=1)': [
        results_base['recall_1'],
        results_poly['recall_1'],
        results_l2['recall_1'],
        results_l1['recall_1'],
        results_xgb['recall_1'],
        results_knn['recall_1'],
        results_svm['recall_1']
    ],
    'F1-Score (PCV=1)': [
        results_base['f1_score_1'],
        results_poly['f1_score_1'],
        results_l2['f1_score_1'],
        results_l1['f1_score_1'],
        results_xgb['f1_score_1'],
        results_knn['f1_score_1'],
        results_svm['f1_score_1']
    ]
}

# Crear el DataFrame de comparación
comparison_df = pd.DataFrame(model_comparison_data)

# Mostrar el DataFrame, ordenando por PR-AUC para ver el mejor modelo
display(comparison_df.sort_values(by='PR-AUC', ascending=False))

## Comparación y selección de modelos.

La tabla comparativa muestra que los modelos presentan desempeños altos y muy cercanos entre sí, lo que indica estabilidad del fenómeno y consistencia de las variables utilizadas. El XGBoost optimizado alcanza el mejor equilibrio global, con el PR-AUC más alto, lo que lo posiciona como la alternativa más adecuada para priorizar personas con riesgo de progresión a PSCV en un contexto preventivo. La regresión logística L1 optimizada exhibe un rendimiento prácticamente equivalente, con mayor precisión, lo que reduce falsos positivos y la hace especialmente útil como modelo interpretable y de respaldo clínico. Los modelos logísticos con polinomios y L2 no aportan mejoras relevantes adicionales, pese a su mayor complejidad. El SVM optimizado logra el mayor recall, detectando más casos verdaderos, pero a costa de mayor sobre-clasificación, lo que podría tensionar la capacidad operativa de APS. La regresión logística base y KNN muestran un desempeño inferior y no se recomiendan para uso operativo. En conjunto, los resultados respaldan el uso de XGBoost como modelo principal y de regresión logística L1 como complemento explicativo, alineados con una gestión eficiente y clínicamente coherente del riesgo cardiovascular en APS.



## **Análisis de Varianza (ANOVA) de una via, para comparar todos los modelos, según la metrica F1-Score**

### Función para el cálculo y recolección de los F-scores para la clase positiva (PCV=1) para cada fold.

In [ ]:
# @title
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from copy import deepcopy

def get_per_fold_f1_scores(model, X, y, cv_splits=5):
    """
    Calcula y recolecta los F1-scores para la clase positiva (PCV=1)
    para cada fold durante la validación cruzada estratificada k-fold.

    Args:
        model: El modelo de machine learning (ej., un Pipeline de sklearn).
        X (pd.DataFrame): El DataFrame de características.
        y (pd.Series): La serie objetivo.
        cv_splits (int): El número de divisiones para la validación cruzada estratificada k-fold.

    Returns:
        list: Una lista de los F1-scores para la clase positiva para cada fold.
    """
    # Inicializa StratifiedKFold para asegurar que la proporción de clases se mantenga en cada fold
    skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=42)
    f1_scores_per_fold = []

    # Itera sobre cada fold de la validación cruzada
    for fold_idx, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        # Divide los datos en conjuntos de entrenamiento y prueba para el fold actual
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Crea una copia profunda del modelo para asegurar un entrenamiento independiente en cada fold
        model_current_fold = deepcopy(model)
        # Entrena el modelo con los datos de entrenamiento del fold actual
        model_current_fold.fit(X_train, y_train)

        # Realiza predicciones de clase (usando un umbral de 0.5 si el modelo predice probabilidades)
        # Para classification_report, predict() directo suele ser suficiente, o usar el umbral en proba
        if hasattr(model_current_fold, 'predict'):
            y_pred = model_current_fold.predict(X_test)
        elif hasattr(model_current_fold, 'predict_proba'):
            y_pred = (model_current_fold.predict_proba(X_test)[:, 1] >= 0.5).astype(int)
        else:
            raise ValueError("El modelo debe tener un método 'predict' o 'predict_proba'")

        # Obtiene el reporte de clasificación y extrae el F1-score para la clase 1 (positiva)
        report = classification_report(y_test, y_pred, output_dict=True)
        f1_score_class_1 = report['1']['f1-score']
        # Agrega el F1-score del fold a la lista
        f1_scores_per_fold.append(f1_score_class_1)

    return f1_scores_per_fold

print("La función de ayuda 'get_per_fold_f1_scores' fue creada exitosamente.")


### Cálculo y recolección de los F-scores para la clase positiva (PCV=1) para cada fold.

In [ ]:
# @title
# Lista de nombres de los modelos que se evaluarán
model_names = [
    'Regresión Logística Base',
    'Regresión Logística con Polinomios',
    'Regresión Logística L2 (Optimizada)',
    'Regresión Logística L1 (Optimizada)',
    'KNN (Optimizado)',
    'SVM (Optimizado)',
    'XGBoost (Optimizado)'
]

# Diccionario que mapea los nombres de los modelos a sus respectivas instancias de Pipeline
# Estas instancias de Pipeline ya han sido entrenadas o configuradas previamente
models = {
    'Regresión Logística Base': log_reg_base,
    'Regresión Logística con Polinomios': log_reg_poly,
    'Regresión Logística L2 (Optimizada)': best_model_l2,
    'Regresión Logística L1 (Optimizada)': best_model_l1,
    'KNN (Optimizado)': best_model_knn,
    'SVM (Optimizado)': best_model_svm,
    'XGBoost (Optimizado)': best_model_xgb
}

# Diccionario para almacenar los F1-scores de la clase positiva (PCV=1) para cada modelo y fold
f1_scores_data = {}
# Itera sobre cada modelo en el diccionario 'models'
for name, model in models.items():
    print(f"Recolectando F1-scores para {name}...")
    # Llama a la función auxiliar para obtener los F1-scores por fold
    # y los almacena en el diccionario f1_scores_data
    f1_scores_data[name] = get_per_fold_f1_scores(model, X, y, cv_splits=5)
    print(f"  F1-scores para {name}: {f1_scores_data[name]}")

# Convierte el diccionario de F1-scores a un DataFrame de pandas para facilitar su manejo y análisis
f1_scores_df = pd.DataFrame(f1_scores_data)
print("\nF1-scores recolectados para todos los modelos:")
display(f1_scores_df)

### Análisis de Varianza (ANOVA) de una via y pruebas de comparación múltiple de promedios (post-hoc de Tukey HSD).

In [ ]:
# @title
from scipy.stats import f_oneway
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Preparar los datos para el análisis ANOVA
# anova_data contendrá una lista de arrays, donde cada array son los F1-scores de un modelo.
anova_data = [f1_scores_df[col].values for col in f1_scores_df.columns]

# Realizar la prueba ANOVA de una vía (One-way ANOVA test)
# Compara las medias de dos o más grupos para determinar si al menos uno de ellos es significativamente diferente.
f_statistic, p_value = f_oneway(*anova_data)

# Imprimir los resultados de la estadística F y el valor p del ANOVA
print(f"ANOVA F-statistic: {f_statistic:.4f}")
print(f"ANOVA p-value: {p_value:.4f}")

# Comprobar la significancia estadística del valor p
# Si p-value < 0.05, indica que hay diferencias significativas entre al menos dos medias de grupo.
if p_value < 0.05:
    print("\nEl valor p de ANOVA es menor que 0.05, lo que indica una diferencia estadísticamente significativa entre los F1-scores promedio de al menos dos modelos.")
    print("Procediendo con la prueba post-hoc de Tukey HSD.")

    # Preparar los datos para la prueba post-hoc de Tukey HSD:
    # Se apilan todos los F1-scores en un solo array y se crean las etiquetas de grupo correspondientes.
    f1_scores_stacked = f1_scores_df.stack().values
    model_labels = f1_scores_df.stack().index.get_level_values(1).values

    # Realizar la prueba post-hoc de Tukey HSD (Honest Significant Difference)
    # Esta prueba se utiliza para realizar comparaciones por pares entre las medias de los grupos
    # después de que la prueba ANOVA ha indicado una diferencia significativa general.
    tukey_results = pairwise_tukeyhsd(endog=f1_scores_stacked, groups=model_labels, alpha=0.05)

    # Imprimir los resultados de la prueba de Tukey HSD
    print("\nResultados de la prueba Post-hoc de Tukey HSD:")
    print(tukey_results)

    # Resumir la interpretación de los resultados de Tukey HSD
    print("\nInterpretación de Tukey HSD:")
    print("La columna 'reject' indica si la hipótesis nula (de que las medias de los dos grupos son iguales) es rechazada. Si es 'True', existe una diferencia estadísticamente significativa entre los F1-scores de ese par de modelos.")

# Si el valor p de ANOVA no es significativo, se concluye que no hay diferencias significativas entre las medias.
else:
    print("\nEl valor p de ANOVA no es menor que 0.05, lo que indica que no hay una diferencia estadísticamente significativa entre los F1-scores promedio de los modelos.")

### Comentario:

Los resultados del Análisis de Varianza (ANOVA) mostraron un valor p de 0.0720. Dado que este valor es mayor que el umbral de significancia de 0.05, concluimos que no hay una diferencia estadísticamente significativa entre los F1-scores promedio de los diferentes modelos evaluados. Esto significa que, con base en esta prueba, no podemos afirmar que un modelo rinda significativamente mejor que otro en términos de F1-score. Por lo tanto, no es necesario realizar una prueba post-hoc de Tukey HSD, ya que no se encontraron diferencias significativas globales que necesiten ser exploradas en pares.

## Determinación del punto de corte (umbral de clasificación)

Para el **modelo XGBoost optimizado** se recalculó la curva ROC a partir de las probabilidades agregadas en los cinco folds de validación cruzada, estimando el **índice de Youden** (sensibilidad + especificidad − 1) para cada posible umbral. El punto de corte óptimo se definió como aquel que maximizó dicho índice, obteniéndose un umbral cercano a 0,52 (probabilidad de pertenecer a PCV=1). Este valor se propone como punto de corte operativo para clasificar a las personas en alto riesgo de progresión a PSCV, balanceando sensibilidad y especificidad en un contexto de priorización preventiva en APS.

In [ ]:
# @title
from sklearn.metrics import roc_curve
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold

# Re-ejecutar la validación cruzada para obtener y_true_all y y_proba_all
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
y_true_all = []
y_proba_all = []

for train_idx, test_idx in skf.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Crear una nueva instancia del best_model_xgb para que se re-entrene en cada fold
    from copy import deepcopy
    model_current_fold = deepcopy(best_model_xgb)
    model_current_fold.fit(X_train, y_train)
    y_proba = model_current_fold.predict_proba(X_test)[:, 1]

    y_true_all.extend(y_test)
    y_proba_all.extend(y_proba)

y_true_all = np.array(y_true_all)
y_proba_all = np.array(y_proba_all)

# Calcular la curva ROC
fpr, tpr, thresholds = roc_curve(y_true_all, y_proba_all)

# Calcular el Índice de Youden
youden_index = tpr - fpr

# Encontrar el umbral óptimo (donde el Índice de Youden es máximo)
optimal_threshold_idx = np.argmax(youden_index)
optimal_threshold = thresholds[optimal_threshold_idx]

print(f"El umbral óptimo (Indice de Youden) para el modelo XGBoost es: {optimal_threshold:.4f}")

# Graficar la curva ROC y marcar el umbral óptimo
plt.figure(figsize=(10, 7))
plt.plot(fpr, tpr, label=f'Curva ROC (AUC = {results_xgb["roc_auc"]:.3f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Clasificador aleatorio')
plt.scatter(
    fpr[optimal_threshold_idx],
    tpr[optimal_threshold_idx],
    marker='o',
    color='red',
    label=f'Umbral Óptimo = {optimal_threshold:.4f}'
)
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR) / Recall')
plt.title('Curva ROC con Umbral Óptimo (XGBoost)')
plt.legend()
plt.grid()
plt.show()

### Modelo Optimizado XGBoost (Extreme Gradient Boosting), Metricas de evaluación y curvas ROC y Precisión-Recall, usando Umbral Óptimo.

In [ ]:
# @title
optimal_threshold = 0.5197
results_xgb_optimized_threshold = evaluate_model_cv(best_model_xgb, X, y, cv_splits=5, threshold=optimal_threshold)


# Actividad 4 — Redes Neuronales (ANN y CNN)

En esta sección se incorporan **Redes Neuronales Artificiales (MLP)** y una **Red Neuronal Convolucional (CNN)**, manteniendo el mismo dataset y el mismo preprocesamiento base para garantizar comparabilidad entre modelos.


## Paso 1. Preparación de datos para redes neuronales

Las redes neuronales requieren entradas numéricas en forma de arreglos (NumPy). Usaremos el mismo `preprocessor_base` (imputación + one-hot + escalamiento) para transformar `X_train` y `X_test`.


In [ ]:
# @title
import numpy as np

# 1) Ajustar el preprocesador SOLO con train (evita leakage)
preprocessor_base.fit(X_train)

# 2) Transformar a matrices numéricas
X_train_nn = preprocessor_base.transform(X_train)
X_test_nn  = preprocessor_base.transform(X_test)

# 3) Asegurar formato float32 (mejor para NN)
X_train_nn = X_train_nn.astype(np.float32)
X_test_nn  = X_test_nn.astype(np.float32)

# 4) Asegurar y en formato (n,1) float32
y_train_nn = np.array(y_train).astype(np.float32).reshape(-1, 1)
y_test_nn  = np.array(y_test).astype(np.float32).reshape(-1, 1)

n_features = X_train_nn.shape[1]
print("n_features:", n_features)
print("X_train_nn:", X_train_nn.shape, " | y_train_nn:", y_train_nn.shape)


## Paso 1. MLP (Perceptrón Multicapa)

Implementamos una red neuronal densa (fully connected) para **clasificación binaria**:
- Capa de entrada acorde a `n_features`
- 1–2 capas ocultas (ReLU)
- Capa de salida con **sigmoide**

Se utiliza **Binary Crossentropy** como función de pérdida y **Adam** como optimizador.


In [ ]:
# @title
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_mlp(input_dim, lr=1e-3, hidden_units=(64, 32), dropout=0.0):
    model = keras.Sequential()
    model.add(layers.Input(shape=(input_dim,)))
    for u in hidden_units:
        model.add(layers.Dense(u, activation="relu"))
        if dropout > 0:
            model.add(layers.Dropout(dropout))
    model.add(layers.Dense(1, activation="sigmoid"))

    opt = keras.optimizers.Adam(learning_rate=lr)
    model.compile(
        optimizer=opt,
        loss="binary_crossentropy",
        metrics=[
            keras.metrics.BinaryAccuracy(name="accuracy"),
            keras.metrics.AUC(name="auc_roc", curve="ROC"),
            keras.metrics.AUC(name="auc_pr", curve="PR")
        ]
    )
    return model

mlp = build_mlp(n_features, lr=1e-3, hidden_units=(64, 32), dropout=0.1)
mlp.summary()


In [ ]:
# @title
import time
import matplotlib.pyplot as plt

early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

t0 = time.perf_counter()
history = mlp.fit(
    X_train_nn, y_train_nn,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=0
)
t_mlp = time.perf_counter() - t0
print(f"Tiempo entrenamiento MLP (s): {t_mlp:.2f}")

# Curvas de entrenamiento
plt.figure()
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.title("MLP — Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()


In [ ]:
# @title
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score,
    roc_curve, precision_recall_curve, ConfusionMatrixDisplay, confusion_matrix
)

# Predicciones en test
y_score_mlp = mlp.predict(X_test_nn, verbose=0).ravel()
y_pred_mlp = (y_score_mlp >= 0.5).astype(int)

print("MLP — TEST")
print("Accuracy:", accuracy_score(y_test, y_pred_mlp))
print("Precision:", precision_score(y_test, y_pred_mlp, zero_division=0))
print("Recall:", recall_score(y_test, y_pred_mlp, zero_division=0))
print("F1:", f1_score(y_test, y_pred_mlp, zero_division=0))
print("AUC-ROC:", roc_auc_score(y_test, y_score_mlp))
print("PR-AUC:", average_precision_score(y_test, y_score_mlp))

# Matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_mlp))
disp.plot(values_format="d")
plt.title("MLP — Confusion Matrix (test)")
plt.show()

# ROC
fpr, tpr, _ = roc_curve(y_test, y_score_mlp)
plt.figure()
plt.plot(fpr, tpr)
plt.plot([0,1],[0,1], linestyle="--")
plt.title("MLP — ROC (test)")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.show()

# Precision-Recall
prec, rec, _ = precision_recall_curve(y_test, y_score_mlp)
plt.figure()
plt.plot(rec, prec)
plt.title("MLP — Precision–Recall (test)")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.show()


## Paso 2. Experimentos: learning rate y batch size

Para analizar estabilidad y convergencia, se prueban:
- **Learning rate**: 0.001 vs 0.0001
- **Batch size**: 16 vs 64

Se reporta desempeño en test y tiempo de entrenamiento.


In [ ]:
# @title
from sklearn.metrics import average_precision_score

def train_eval_mlp(lr, batch_size, epochs=50):
    model = build_mlp(n_features, lr=lr, hidden_units=(64, 32), dropout=0.1)
    early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

    t0 = time.perf_counter()
    hist = model.fit(
        X_train_nn, y_train_nn,
        validation_split=0.2,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stop],
        verbose=0
    )
    t_train = time.perf_counter() - t0

    y_score = model.predict(X_test_nn, verbose=0).ravel()
    y_pred = (y_score >= 0.5).astype(int)

    metrics = {
        "lr": lr,
        "batch_size": batch_size,
        "epochs_ran": len(hist.history["loss"]),
        "time_s": t_train,
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, zero_division=0),
        "recall": recall_score(y_test, y_pred, zero_division=0),
        "f1": f1_score(y_test, y_pred, zero_division=0),
        "auc_roc": roc_auc_score(y_test, y_score),
        "pr_auc": average_precision_score(y_test, y_score),
    }
    return metrics

experiments = []
for lr in [1e-3, 1e-4]:
    for bs in [16, 64]:
        experiments.append(train_eval_mlp(lr=lr, batch_size=bs))

import pandas as pd
exp_df = pd.DataFrame(experiments).sort_values(["pr_auc","f1"], ascending=False)
exp_df


# Paso 3. Red Neuronal Convolucional

Aunque el dataset es tabular, para fines didácticos se transforma el vector de features en una representación tipo *secuencia* `(n_features, 1)` y se implementa una **Conv1D + Pooling + Dense**. Esto permite discutir el rol de **kernels**, **stride** y **pooling**.

La CNN se incluye con fines exploratorios y didácticos, dado que el dataset no posee una estructura espacial inherente.


### Red Neuronal Convolucional (CNN)

a) Transformación del dataset a representación matricial

En este trabajo, el dataset es tabular (n_features = 8) y no corresponde a imágenes. Para poder aplicar una CNN, se construye una representación matricial simple tratando las variables como una secuencia 1D, donde cada registro se reordena desde:

Entrada MLP: (n_features,)

Entrada CNN: (n_features, 1)

Esto se implementa reshaping como:

X_train_cnn.shape = (n_samples, n_features, 1)

X_test_cnn.shape = (n_samples, n_features, 1)

Esta estrategia permite aplicar convoluciones 1D sobre la “vecindad” de features, aunque se reconoce que dicha vecindad no es espacial real como en imágenes.

b) CNN implementada (Conv + Pooling + Dense)

Se implementa una CNN básica con la siguiente estructura mínima:

Capa convolucional (Conv1D): extrae patrones locales sobre la secuencia de features.

Pooling (MaxPooling1D): reduce dimensionalidad, estabiliza el aprendizaje y disminuye parámetros.

Flatten: convierte el mapa de activaciones en un vector.

Capa densa final (Dense + Sigmoid): entrega probabilidad para clasificación binaria.

c) Rol de kernels, stride y pooling

Kernel (filtro/convolución): define la “ventana” que recorre la entrada. Permite aprender combinaciones locales de variables (por ejemplo, interacciones entre features cercanas en la secuencia).

Stride (paso): determina cuánto se desplaza el kernel en cada movimiento. Stride más grande reduce más rápido la resolución, pero puede perder información; stride pequeño conserva más detalle, pero aumenta costo computacional.

Pooling (submuestreo): reduce la dimensión de los mapas de activación (ej. MaxPooling), ayudando a disminuir sobreajuste y a mejorar eficiencia. En este caso, además compensa el tamaño pequeño de la entrada.

In [ ]:
# @title
# Transformación a formato 3D para Conv1D: (muestras, pasos=n_features, canales=1)
X_train_cnn = X_train_nn.reshape((-1, n_features, 1))
X_test_cnn  = X_test_nn.reshape((-1, n_features, 1))

print("X_train_cnn:", X_train_cnn.shape)


In [ ]:
# @title
def build_cnn(input_steps, lr=1e-3, n_filters=32, kernel_size=3):
    model = keras.Sequential([
        layers.Input(shape=(input_steps, 1)),
        layers.Conv1D(filters=n_filters, kernel_size=kernel_size, activation="relu", padding="same"),
        layers.MaxPooling1D(pool_size=2),
        layers.Flatten(),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ])
    opt = keras.optimizers.Adam(learning_rate=lr)
    model.compile(
        optimizer=opt,
        loss="binary_crossentropy",
        metrics=[
            keras.metrics.BinaryAccuracy(name="accuracy"),
            keras.metrics.AUC(name="auc_roc", curve="ROC"),
            keras.metrics.AUC(name="auc_pr", curve="PR")
        ]
    )
    return model

cnn = build_cnn(n_features, lr=1e-3, n_filters=32, kernel_size=3)
cnn.summary()


In [ ]:
# @title
early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

t0 = time.perf_counter()
history_cnn = cnn.fit(
    X_train_cnn, y_train_nn,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=0
)
t_cnn = time.perf_counter() - t0
print(f"Tiempo entrenamiento CNN (s): {t_cnn:.2f}")

plt.figure()
plt.plot(history_cnn.history["loss"], label="train")
plt.plot(history_cnn.history["val_loss"], label="val")
plt.title("CNN — Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()


In [ ]:
# @title
# Predicciones en test
y_score_cnn = cnn.predict(X_test_cnn, verbose=0).ravel()
y_pred_cnn = (y_score_cnn >= 0.5).astype(int)

print("CNN — TEST")
print("Accuracy:", accuracy_score(y_test, y_pred_cnn))
print("Precision:", precision_score(y_test, y_pred_cnn, zero_division=0))
print("Recall:", recall_score(y_test, y_pred_cnn, zero_division=0))
print("F1:", f1_score(y_test, y_pred_cnn, zero_division=0))
print("AUC-ROC:", roc_auc_score(y_test, y_score_cnn))
print("PR-AUC:", average_precision_score(y_test, y_score_cnn))

disp = ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_cnn))
disp.plot(values_format="d")
plt.title("CNN — Confusion Matrix (test)")
plt.show()


## Paso 4. Comparación final

Se consolidan resultados de modelos clásicos (logística / Random Forest / SVM) y redes neuronales (MLP / CNN) para discutir desempeño, interpretabilidad y costo computacional.


In [ ]:
# @title
# Recalcular predicciones SVM con el X_test actual
y_pred_svm = best_svm.predict(X_test)
y_score_svm = best_svm.predict_proba(X_test)[:, 1]

print("len(y_pred_svm):", len(y_pred_svm))
print("len(y_score_svm):", len(y_score_svm))


In [ ]:
# @title
import numpy as np
import pandas as pd

def check_lengths(name, y_true, y_pred, y_score):
    lt, lp, ls = len(y_true), len(y_pred), len(y_score)
    print(f"{name}: y_true={lt}, y_pred={lp}, y_score={ls}")
    assert lt == lp == ls, f"❌ Largo inconsistente en {name}"

check_lengths("SVM", y_test, y_pred_svm, y_score_svm)
check_lengths("MLP", y_test, y_pred_mlp, y_score_mlp)
check_lengths("CNN", y_test, y_pred_cnn, y_score_cnn)

comparison = pd.DataFrame([
    {"Modelo":"SVM (best grid)",
     "Accuracy": accuracy_score(y_test, y_pred_svm),
     "F1": f1_score(y_test, y_pred_svm, zero_division=0),
     "AUC-ROC": roc_auc_score(y_test, y_score_svm),
     "PR-AUC": average_precision_score(y_test, y_score_svm),
     "Tiempo(s)": t_svm},
    {"Modelo":"ANN (MLP)",
     "Accuracy": accuracy_score(y_test, y_pred_mlp),
     "F1": f1_score(y_test, y_pred_mlp, zero_division=0),
     "AUC-ROC": roc_auc_score(y_test, y_score_mlp),
     "PR-AUC": average_precision_score(y_test, y_score_mlp),
     "Tiempo(s)": t_mlp},
    {"Modelo":"CNN (Conv1D)",
     "Accuracy": accuracy_score(y_test, y_pred_cnn),
     "F1": f1_score(y_test, y_pred_cnn, zero_division=0),
     "AUC-ROC": roc_auc_score(y_test, y_score_cnn),
     "PR-AUC": average_precision_score(y_test, y_score_cnn),
     "Tiempo(s)": t_cnn},
]).sort_values(["PR-AUC","F1"], ascending=False)

comparison



## Paso 4. Comparación final y análisis crítico

- **Desempeño predictivo**: ¿MLP o CNN mejora respecto a SVM / Random Forest?
- **Interpretabilidad**: modelos clásicos (ej. logística) suelen ser más interpretables que ANN/CNN.
- **Costo computacional**: comparar tiempos (`Tiempo(s)`) y estabilidad de entrenamiento.
- **Escalabilidad y datos**: redes neuronales suelen beneficiarse de mayor volumen de datos y/o estructura (imágenes/secuencias).
- **Pertinencia APS rural**: balance entre desempeño, transparencia y facilidad de despliegue.


### 1. Comparación de  los modelos: Regresión logística, Random Forest, SVM, Red neuronal artificial y Red neuronal convolucional.

### Comparación de modelos:

| Modelo                         | Desempeño predictivo | Interpretabilidad | Costo computacional | Comentario |
|--------------------------------|----------------------|-------------------|---------------------|------------|
| Regresión Logística            | Medio                | Alta              | Bajo                | Modelo base, simple y fácilmente interpretable, con menor capacidad para capturar relaciones no lineales. |
| Random Forest                  | Alto                 | Media             | Medio               | Mejora el desempeño al modelar no linealidades e interacciones; permite interpretación parcial mediante importancia de variables. |
| SVM (best grid)                | Alto                 | Media–baja        | Alto                | Presenta uno de los mejores desempeños predictivos, pero con alto tiempo de entrenamiento y menor escalabilidad. |
| Red Neuronal Artificial (MLP)  | Alto                 | Baja              | Bajo–medio          | Desempeño comparable a SVM y Random Forest, con menor costo computacional; adecuada para datos tabulares. |
| Red Neuronal Convolucional (CNN) | Medio–alto           | Baja              | Medio–alto          | No muestra mejoras claras respecto al MLP; su aporte es principalmente didáctico dada la naturaleza tabular del dataset. |

Comentario final

En este problema de predicción de riesgo cardiovascular con datos tabulares de APS, los modelos tradicionales bien ajustados (Random Forest y SVM) y la red neuronal artificial (MLP) presentan desempeños similares. El MLP destaca como una alternativa eficiente, al combinar buen desempeño predictivo con menor costo computacional que el SVM. En contraste, la CNN no aporta ventajas significativas, lo cual es esperable dada la ausencia de una estructura espacial en los datos. En términos aplicados, la elección del modelo debe equilibrar desempeño, interpretabilidad clínica y viabilidad operativa.

### 2. Analisis de diferencias en: Desempeño predictivo, interpretabilidad, costo computacional y escalabilidad.

### Análisis de diferencias entre modelos

**Desempeño predictivo.**
Los modelos evaluados presentan desempeños similares en términos globales. SVM, Random Forest y la red neuronal artificial (MLP) alcanzan los mejores resultados en métricas como F1-score y AUC-ROC. La CNN no muestra una mejora significativa respecto al MLP ni a los modelos tradicionales, lo cual es consistente con la naturaleza tabular del dataset. La regresión logística presenta el menor desempeño, aunque sigue siendo un punto de referencia válido.

**Interpretabilidad.**
La regresión logística es el modelo más interpretable, permitiendo una lectura directa del efecto de cada variable. Random Forest ofrece una interpretabilidad intermedia mediante medidas de importancia de variables. En contraste, SVM y las redes neuronales (MLP y CNN) presentan menor interpretabilidad, ya que su estructura dificulta la explicación directa de las decisiones del modelo, aspecto relevante en contextos clínicos.

**Costo computacional.**
Existen diferencias claras en el costo computacional entre modelos. La regresión logística y el MLP presentan tiempos de entrenamiento bajos. Random Forest muestra un costo intermedio, mientras que el SVM optimizado presenta el mayor tiempo de entrenamiento, especialmente al utilizar validación cruzada y búsqueda de hiperparámetros. La CNN requiere mayor costo que el MLP debido a su arquitectura, sin entregar una mejora proporcional en desempeño.

**Escalabilidad.**
Desde el punto de vista de la escalabilidad, la regresión logística y el MLP son los modelos más adecuados para aumentar el tamaño del dataset sin un incremento significativo en el tiempo de cómputo. Random Forest escala razonablemente, aunque con mayor consumo de recursos. SVM y CNN presentan mayores limitaciones de escalabilidad, especialmente en contextos con mayor volumen de datos o restricciones computacionales, como la APS.

## **Análisis integrado sobre el uso de redes neuronales**

Las redes neuronales aportan ventajas claras frente a modelos clásicos principalmente en escenarios donde existen relaciones altamente no lineales, interacciones complejas entre variables y grandes volúmenes de datos, o cuando los datos presentan una estructura explícita (imágenes, señales, series temporales). En datasets tabulares pequeños o medianos, como el utilizado en este estudio, su ventaja en desempeño predictivo suele ser limitada frente a modelos clásicos bien ajustados.

En términos de riesgo de sobreajuste, las redes neuronales son más propensas a ajustarse al ruido del conjunto de entrenamiento, especialmente cuando el número de parámetros es elevado en relación con la cantidad de datos disponibles. Este riesgo puede mitigarse mediante técnicas como regularización, early stopping, control del número de capas y neuronas, y validación adecuada; sin embargo, sigue siendo un aspecto crítico a considerar en contextos con datos limitados.

La cantidad de datos influye de manera decisiva en la elección del modelo. Los modelos clásicos, como regresión logística, Random Forest y SVM, tienden a funcionar mejor y de forma más estable en conjuntos de datos pequeños o medianos. Las redes neuronales, en cambio, requieren mayores volúmenes de datos para explotar plenamente su capacidad de representación y generalización.

Desde una perspectiva de negocio y aplicada a APS, el uso de redes neuronales no resulta estrictamente necesario para este problema. Si bien el MLP logra un desempeño comparable a los mejores modelos clásicos, no entrega una mejora sustantiva que justifique un aumento en complejidad y menor interpretabilidad. En este contexto, los modelos clásicos bien ajustados continúan ofreciendo un mejor equilibrio entre desempeño predictivo, explicabilidad, costo computacional y viabilidad operativa. Las redes neuronales pueden considerarse como una alternativa complementaria o exploratoria, más que como la solución principal.